In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import collections
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import colors
import matplotlib.image as mpimg
from tqdm import tqdm
from typing import List, Tuple, Dict, Any
import gc

In [ ]:
def draw_3_linesplots_next_to_each_other(
    data: pd.DataFrame,
    keypoint_thresholds:List[int],
    detector_names:List[str],
    set_names:List[str],
    val_column_id:str ,
    color_scheme:Dict,
    marker_scheme:Dict,
    id2name:Dict,
    xlabel:str='',
    ylabel:str='',
    legend_options:Dict={
        'loc':'upper left', 
        'bbox_to_anchor': (0, 1.)
    }) -> mpl.figure.Figure:
    
    fig, axes = plt.subplots(nrows=1, ncols=3)
    for i, kp_thresh in enumerate(keypoint_thresholds):
        ax = axes[i]

        ax.tick_params(
        axis='both',         # changes apply to the axis
        which='both',        # both major and minor ticks are affected
        bottom=True,        # ticks along the bottom edge are off (False) or on (True)
        left=True)          # ticks along left edge are off (False) or on (True)

        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False) 

        ax.tick_params(axis='x', which='both', labelrotation=30) # Rotate x-axis label

        ax.set_ylim(0, 100) # Y-Axis height forced from 0 to 1

        #ax.grid(linestyle='-', linewidth='0.5', color='red')
        ax.grid(linestyle=':')
        
        ax.set_title(kp_thresh)

        ylabels = [id2name[x] for x in set_names]

        for j, detector_name in enumerate(detector_names):


            ys = data[(data.keypoint_threshold == kp_thresh) &
                     (data.detector_name == detector_name)][val_column_id].values * 100
            ax.plot(
                ylabels, 
                ys,
                linestyle=line_styles[2],
                color=color_scheme[detector_name],
                alpha=0.3)

            ax.scatter(
                ylabels,
                ys,
                color=color_scheme[detector_name],
                marker=marker_scheme[detector_name],
                label=id2name[detector_name])


        if i == 0:
            # Show legends only inside first subplot.
            #ax.legend(loc='upper left', bbox_to_anchor=(0, 1.))
            ax.legend(**legend_options)

            # Show y label in first plot.
            ax.set_ylabel(ylabel, fontsize=12)

        if i == 1:
            # Set  x label only in the center plot.
            ax.set_xlabel(xlabel, fontsize=12)
            
    return fig

# Plot statistics of Detector Evaluation

In [ ]:
#####################
### FUNCTIONS
#####################

def save_figure(
  path_output:str,
  fig_name:str, 
  figure: mpl.figure.Figure,
  dpi:int=300,
  tight_layout:bool=False) -> None:

    if not os.path.exists(path_output):
        os.makedirs(path_output, exist_ok=True)

    f_out = os.path.join(path_output, fig_name)
  
    if tight_layout:
        figure.savefig(f_out, bbox_inches='tight', dpi=dpi)
    else:
        figure.savefig(f_out, dpi=dpi)

def get_num_of_found_kpts(
    df:pd.DataFrame,
    kp_thresh:int) -> List[int]:
    # Get names of all images j and the first of image_i to 
    # get all image names.
    frame_j = df[['image_j', 'num_kpts_j']].values
    frame_i = df[['image_i', 'num_kpts_i']].values[0].reshape(1, 2)

    # Stack them, find duplicates in name and remove them, to get
    # list of all images in set.
    frame = np.vstack([frame_i, frame_j])
    _, dup_idx = np.unique(frame[:, 0], return_index=True)
    frame = frame[dup_idx][:, 1] # only number of keypoints

    # Find values that are greater than current kp_thresh and set
    # them to kp_thresh.
    gt_thresh_idx = (frame > kp_thresh)
    frame[gt_thresh_idx] = kp_thresh

    # finally find the avg. number of keypoints found for current kp_thresh level.
    #avg_found_kpts = np.mean(frame)
    #return avg_found_kpts
    return frame

############################
### SETTINGS
############################

id2name = {
    'sift': 'SIFT',
    'lift': 'LIFT',
    'tilde': 'TILDE',
    'superpoint': 'SuperPoint',
    'tcovdet': 'TCovDet',
    'tfeat': 'TFeat',
    'doap': 'DOAP',
    'chamonix': 'Chamonix', 
    'courbevoie': 'Courbevoie', 
    'frankfurt': 'Frankfurt',
    'mexico': 'Mexico',
    'panorama': 'Panorama', 
    'stlouis': 'St. Louis'
}

color_scheme = {
  'sift': '#1f77b4',
  'lift': '#ff7f0e',
  'tilde': '#2ca02c',
  'superpoint': '#d62728',
  'tcovdet': '#9467bd',
  'tfeat': '#2ca02c',
  'doap': '#9467bd'
}

keypoint_threshold_colors = ['skyblue', 'crimson', 'purple']

marker_scheme = {
  'sift': 'o',
  'lift': 'x',
  'tilde': 'd',
  'superpoint': '^',
  'tcovdet': 'h',
  'tfeat': '*',
  'doap': 'p'
}

line_styles = ['-', '-.', ':', '--']


collection_name = 'webcam'
file_scheme = '_{}.csv'.format(collection_name)

data_dir = '/home/mizzade/Workspace/diplom/outputs/eval_detectors'
output_dir = '/home/mizzade/Workspace/diplom/outputs/eval_detectors/plots'
file_name = 'repeatability_webcam.csv'

############################
### MAIN
############################
save_outputs = True

# Load data
df = pd.read_csv(os.path.join(data_dir, file_name), sep=',', comment='#')

keypoint_thresholds = sorted(df.keypoint_threshold.unique())
set_names = sorted(df.set_name.unique())
detector_names = sorted(df.detector_name.unique())

In [ ]:
df.head()

### 1) Find average repeatability and accuracy of matched keypoints for each detector partitioned by the image sets.

In [ ]:
### 1. Group the dataframe for easy value extraction

# 1.1 Take only essential information
_df = df[['keypoint_threshold', 'detector_name', 'set_name', 'repeatability', 'accuracy']]

# 1.2 Group by keypoint threshold, detector name and finally the set name
_df = _df.groupby(['keypoint_threshold', 'detector_name','set_name', ], as_index=False)

# 1.3 Find the mean values for each grouping. A group is (kp_thresh, detector, set).
_df = _df.aggregate(np.mean)

Draw Repeatability for maximanl number of keypoints $kp_{thresh} \in [1000, 5000, 10000]$

In [ ]:
plt.rcParams['figure.figsize'] = [16, 5]
fig = draw_3_linesplots_next_to_each_other(
    _df,
    keypoint_thresholds,
    detector_names,
    set_names,
    'repeatability',
    color_scheme,
    marker_scheme,
    id2name,
    ylabel='Wiederholbarkeit in %',
    xlabel='Bildersets')

fig.suptitle('Wiederholbarkeit der Keypoints bei maximal 1000, 5000 und 10000 Keypoints', fontsize=14);
fig.tight_layout()
fig.subplots_adjust(top=0.85)

if save_outputs:
    save_figure(output_dir,'rep_of_found_pts_per_threshold_set_and_detector.png', fig)

Draw thes same for accuracy

In [ ]:
plt.rcParams['figure.figsize'] = [16, 5]
fig = draw_3_linesplots_next_to_each_other(
    _df,
    keypoint_thresholds,
    detector_names,
    set_names,
    'accuracy',
    color_scheme,
    marker_scheme,
    id2name,
    ylabel='Genauigkeit in %',
    xlabel='Bildersets',
    legend_options={
        'loc':'lower left', 
        'bbox_to_anchor': (0, 0)})

fig.suptitle('Genauigkeit der wiederholbaren Keypoints bei maximal 1000, 5000 und 10000 Keypoints', fontsize=14);
fig.tight_layout()
fig.subplots_adjust(top=0.85)

if save_outputs:
    save_figure(output_dir,'acc_of_found_pts_per_threshold_set_and_detector.png', fig)

### 2) Show the average number of found keypoints  in the collection partitioned by detector.

In [ ]:
# Build a new dataframe, containing: keypoint_threshold, detector_name, set_name, avg_num_kpts_found, num_images
df2 = pd.DataFrame(columns=['keypoint_threshold', 'detector_name', 'set_name', 'avg_num_kpts', 'num_images'])

_g = df.groupby(['keypoint_threshold', 'detector_name','set_name', ], as_index=False)
for name, group in _g:
    keypoint_threshold, detector_name, set_name = name
    
    # To get the number of found keypoints, first find the number of unique image names in set, called N.
    u_image_names = np.unique(group[['image_i', 'image_j']].values)
    num_images = len(u_image_names)
    
    
    # Then take the first N-1 elements. `num_kpts_j` will have the number of keypoints in the N-1 images
    # missing only the first image. We add this value by taking num_kpts_i of the first entry.
    _p1 = group.iloc[0][['num_kpts_i']].values # number of keypoints for first image in set
    _p2 = group[:(num_images-1)][['num_kpts_j']].values.ravel() # number of keypoitns in remaining images in set.
    num_kpts = np.append(_p1, _p2)
    
    # Clip num keypoints according to keypoint threshold
    num_kpts = np.clip(num_kpts, 0, keypoint_threshold)
    
    df2 = df2.append({
        'keypoint_threshold': keypoint_threshold,
        'detector_name': detector_name,
        'set_name': set_name,
        'avg_num_kpts': np.mean(num_kpts),
        'num_images': num_images
    }, ignore_index=True)

In [ ]:
plt.rcParams['figure.figsize'] = [16,12]
fig, axes = plt.subplots(3, 1)
axes = axes.reshape(3, 1)

xs = [id2name[x] for x in set_names]

for i, kp_threshold in enumerate(keypoint_thresholds):
    ax = axes[i][0]
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(linestyle=':', alpha=0.3)

    x_labels = xs
    N = len(x_labels)        # number of ticks
    M = len(detector_names)  # number of bars per tick
    D = 3                    # distance between ticks

    # Tick position values
    ticks = np.arange(N) * D

    # Width of one bar:
    width_of_bar = 0.5 * D / M

    # margin after each bar:
    margin_of_bar = 0.5 * width_of_bar

    # Set labels for each tick
    ax.set_xticklabels(x_labels)

    # Width of a "block" of bars.
    width_of_block = M * width_of_bar + (M-1) * margin_of_bar

    # Set position of ticks
    ax.set_xticks(ticks + width_of_block * 0.5 - width_of_bar * 0.5 - margin_of_bar)

    for num_of_tick, set_name in enumerate(set_names):        
        for num_of_bar, detector_name in enumerate(detector_names):
            _fcolor = list(colors.to_rgba(color_scheme[detector_name]))
            _fcolor[3] = 0.3 # set alpha to 0.3
            _ecolor = colors.to_rgba(color_scheme[detector_name])

            labeltext = ''
            if num_of_tick == 0:
                labeltext = '{}'.format(id2name[detector_name])
                
            ys = df2[(df2.keypoint_threshold == kp_threshold) &
                (df2.detector_name == detector_name) &
                (df2.set_name == set_name)].avg_num_kpts.values
            
            _x = ticks[num_of_tick] + num_of_bar * width_of_bar + (num_of_bar -1) * margin_of_bar,

            ax.bar(
                ticks[num_of_tick] + num_of_bar * width_of_bar + (num_of_bar -1) * margin_of_bar,
                ys,
                width_of_bar,
                facecolor=_fcolor,
                edgecolor=_ecolor,
                label=labeltext)

    if i == 0:
        # Legend only for the first subplot
        ax.legend(loc='lower right', bbox_to_anchor=(1.0, 1.01))      

if save_outputs:
    save_figure(output_dir,'avg_num_kpts_per_threshold_set_and_detector.png', fig)

### Get the average number of found keypoints for the three keypoint thresholds paritioned by detectors

In [ ]:
plt.rcParams['figure.figsize'] = [16,4]
fig, axes = plt.subplots(1, 2, gridspec_kw={'width_ratios': [3, 1]})

ax = axes[0]
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(linestyle=':', alpha=0.3)


xs = [id2name[x] for x in detector_names]

x_labels = xs
N = len(x_labels)             # number of ticks
M = len(keypoint_thresholds)  # number of bars per tick
D = 3                         # distance between ticks

# Tick position values
ticks = np.arange(N) * D

# Width of one bar:
width_of_bar = 0.5 * D / M

# margin after each bar:
margin_of_bar = 0.5 * width_of_bar

# Set labels for each tick
ax.set_xticklabels(x_labels)

# Width of a "block" of bars.
width_of_block = M * width_of_bar + (M-1) * margin_of_bar

# Set position of ticks
ax.set_xticks(ticks + width_of_block * 0.5 - width_of_bar * 0.5 - margin_of_bar)

# For all keyoint thresholds draw a line

handles_for_thresholds = []
for i, kp_threshold in enumerate(keypoint_thresholds):
    # Plot the keypoint threshold limit
#   ax.axhline(t, color='lightslategrey', alpha=0.3, linestyle=line_styles[2])
#   ax.text(5.35, t, 'max', color='lightslategrey')
    l = ax.axhline(kp_threshold, 
               color=keypoint_threshold_colors[i], 
               alpha=0.4, 
               linestyle=line_styles[3],
               label='{} Keypoints'.format(kp_threshold))
    
    handles_for_thresholds.append(l)
    
# Create first legend for the threshold values
thresh_legend = ax.legend(handles=handles_for_thresholds, loc='upper right', bbox_to_anchor=(1, 1))

# Add legend manually to the current axes
ax.add_artist(thresh_legend)    
    
# Collect handels for second legend here
handles_for_bars = []

for num_of_tick, detector_name in enumerate(detector_names):        
    for num_of_bar, kp_threshold in enumerate(keypoint_thresholds):
        _fcolor = list(colors.to_rgba(color_scheme[detector_name]))
        _fcolor[3] = 0.3 # set alpha to 0.3
        _ecolor = colors.to_rgba(color_scheme[detector_name])

        labeltext = ''
        if num_of_bar == 0:
            labeltext = '{}'.format(id2name[detector_name])

        ys = df2[(df2.keypoint_threshold == kp_threshold) &
            (df2.detector_name == detector_name)]
        
        # get weigthed average number of found keypoints over all sets
        # for this detector at this keypoint threshold
        total_num_images = np.sum(ys.num_images)
        weights = ys.num_images / total_num_images
        weighted_average = ys.avg_num_kpts * weights
        avg_kpts_found = np.sum(weighted_average)

        h = ax.bar(
            ticks[num_of_tick] + num_of_bar * width_of_bar + (num_of_bar -1) * margin_of_bar,
            avg_kpts_found,
            width_of_bar,
            facecolor=_fcolor,
            edgecolor=_ecolor,
            label=labeltext)
        
        if num_of_bar == 0:
            handles_for_bars.append(h)
            

# Create second legend
bar_legend = ax.legend(handles=handles_for_bars, loc='upper left', bbox_to_anchor=(0, 1))      
# Add manually
ax.add_artist(bar_legend)


# Build right site heatmap
ax = axes[1]
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

ax.yaxis.tick_right() # Y-Tick labels on right side

ax.tick_params(
    axis='both',         # changes apply to the axis
    which='both',        # both major and minor ticks are affected
    bottom=True,        # ticks along the bottom edge are off (False) or on (True)
    right=False,          # ticks along left edge are off (False) or on (True)
    left=False)

ax.set_yticklabels([''] + [id2name[x] for x in detector_names])

ax.tick_params(axis='x', which='both', labelrotation=30) # Rotate x-axis label
ax.set_xticklabels([''] + keypoint_thresholds) # This label bug...


scoremap = []
for num_of_tick, detector_name in enumerate(detector_names):
    
    ys_per_threshold = []
    for num_of_bar, kp_threshold in enumerate(keypoint_thresholds):

        ys = df2[(df2.keypoint_threshold == kp_threshold) &
            (df2.detector_name == detector_name)]
        
        # get weigthed average number of found keypoints over all sets
        # for this detector at this keypoint threshold
        total_num_images = np.sum(ys.num_images)
        weights = ys.num_images / total_num_images
        weighted_average = ys.avg_num_kpts * weights
        avg_kpts_found = np.sum(weighted_average)
    
        # Add value to list
        ys_per_threshold.append(avg_kpts_found)
    
    # Add list of values to list
    scoremap.append(ys_per_threshold)
scoremap = np.array(scoremap)   
    
    
# Build custom color map, so that each row has the 
# color of the corresponding detector.

# Get the corresponding color for each detector as [r,g,b,a] list
_colors = [list(colors.to_rgba(color_scheme[x])) for x in detector_names]

# Set each color's alpha to 0.3
for _c in _colors:
    _c[3] = 0.3
    
# From that, create a color map
cmap_custom = mpl.colors.LinearSegmentedColormap.from_list("", _colors, len(detector_names))

# Create a colormap map, so the heatmap has the colors at the correct positions
colormap = np.zeros((len(detector_names), len(keypoint_thresholds))) + np.arange(len(detector_names))[:, np.newaxis]
    
ax.imshow(colormap, cmap=cmap_custom)
for i in range(len(detector_names)):
    for j in range(len(keypoint_thresholds)):
        t = '{:.1f}'.format(scoremap[i, j])
        text = ax.text(j, i, 
                       t, 
                       ha='center', 
                       va='center', 
                       color='black');
       
        
        
fig.tight_layout();

if save_outputs:
    save_figure(output_dir,'avg_num_kpts_per_detector_and_thresh.png', fig)
